In [333]:
import xlwings as xw
import pandas as pd
import json

In [334]:
wb = xw.Book('Matrix.xlsx')
#wb = xw.Book(r'C:\Users\Porter\OneDrive - Danmarks Tekniske Universitet\Christian Bertram\MDM.xlsx')

In [335]:
totalmdm = wb.sheets['Sheet1']

In [336]:
mdmdata = totalmdm.range('A1:EB133').value
metadata = totalmdm.range('EC5:EF133').value
metadata[120]

[u'Izokee Connector set',
 u'2,54 mm JST XHP JST Connector Kit \u2013 Pin Header 2,54 mm',
 None,
 None]

In [337]:
#get nodes
mdmnodes = mdmdata[2] #first row
for node in mdmnodes:
    if not node:
        print node

In [338]:
#assign levels to nodes
newnodes = []
for index, mdmnode in enumerate(mdmnodes):
    node = {}
    if (mdmnode != 'None') and (index != 0) and mdmnode:
        node["name"] = mdmnode
        node["level"] = mdmdata[1][index]
        node["index"] = index - 3
        node["children"] = []
        node["desc"] = noneToSpace(metadata[index-3][1])
        node["imglink"] = noneToSpace(metadata[index-3][2])
        node["prodlink"] = noneToSpace(metadata[index-3][3])
        print node
        newnodes.append(node)

In [339]:
#map node names to indices
nodesdict = {}
for node in newnodes:
    nodesdict[node["name"]] = node["index"]
nodesdict

{u'1-2 Bore 0.770 Pattern Set Screw Hubs': 91,
 u'1-2 Bore External Retaining Rings': 97,
 u'1-2 Bore, Face Thru-Hole Pillow Block': 89,
 u'1-2 in. Diameter Steel Shaft stock, 1-8 in. keyway, 30 in. long': 95,
 u'1-8x1-8x0700 Machine Key, steel': 96,
 u'10-24x3-4 Thread Forming Screw': 98,
 u'150mm x 2 Caster Wheel': 99,
 u'500 Key Hub': 94,
 u'6 HiGrip Rubber Wheel': 93,
 u'6-32 Zinc-Plated Socket Head Machine Screws (25 pack)': 92,
 u'90\xb0 Angle Aluminium Channel Bracket': 87,
 u'90\xb0 Dual Side Mount C (2 pack)': 88,
 u'Abdeckkappe 5 20x20': 65,
 u'Actobotics 1-2 Ball Bearing Quad Pillow Block': 86,
 u'Actobotics 16T Gearmotor Pinion Gear (1-8)': 84,
 u'Actobotics 80T Aluminum Hub Gear (1-2)': 85,
 u'Aluminum Motor Mount C': 90,
 u'Angle Stainless Steel Corner Brackets': 68,
 u'Anti-shake system': 54,
 u'Anti-theft system': 43,
 u'Arrival notification': 58,
 u'Automatic routing': 24,
 u'Battery': 73,
 u'Battery low voltage alarm': 74,
 u'Beer opener': 56,
 u'Body': 62,
 u'Builder

In [340]:
def noneToSpace(inputstring):
    if inputstring:
        return inputstring
    else:
        return u'None'

In [341]:
def attachChildren(startindex):
    print "attachChildren of node: ", newnodes[startindex]['name'] 
    #startindex = node.index
    startnode = newnodes[startindex]
    children = findChildren(startindex)
    print "returned children are: ", children
    startnode["children"].append(children)
    for child in children:
        attachChildren(nodesdict[child])
    return

In [342]:
#find immediate children
#iterate through next nodes / add to resultarray if node.level is next to startnode level
#if node.level equals start node level, we exit the function
def findChildren(startindex):
    resultarray = []
    print "Returning children for node ", newnodes[startindex] 
    startlevel = newnodes[startindex]["level"]
    for node in newnodes[startindex+1:]:
        print "checking node %s with index %d and level %d and startindex %d" % (node["name"], node["index"], node["level"], startindex)  
        if node["level"] == startlevel + 1:
            resultarray.append(node["name"])
            print node['name']
        elif node["level"] == startlevel:
            print "samelevel reached"
            return resultarray
    return resultarray

In [343]:
#find top level nodes
lvl1nodes = []
for node in newnodes:
    if node["level"] == 1:
        lvl1nodes.append(node["name"])
lvl1nodes

[u'USECASES', u'STAKEHOLDERS', u'REQUIREMENTS', u'STRUCTURE']

In [344]:
#attachChildren(0)
#for nodes that have level one, run attachChildren
for lvl1node in lvl1nodes:
    attachChildren(nodesdict[lvl1node])

In [345]:
#assign domains based on the lvl1node
for index, node in enumerate(newnodes):
    if node["level"] == 1:
        node["domain"] = node['name']
        otherindex = index + 1
        print index
        while newnodes[otherindex]["level"] != 1:
            newnodes[otherindex]["domain"] = node["domain"]
            print newnodes[otherindex]
            otherindex = otherindex + 1
            if otherindex == len(newnodes):
                break;

In [346]:
newnodes

[{'children': [[u'Loading of beverages',
    u'Unloading of beverages',
    u'Driving with load',
    u'General misuse',
    u'Driving unloaded back',
    u'Transporting Pluto',
    u'Charging Pluto',
    u'Disassembled transport',
    u'Demonstration outside',
    u'Test, Service, Repair',
    u'Error handling case',
    u'Demonstration, Show inside']],
  'desc': u'None',
  'domain': u'USECASES',
  'imglink': u'None',
  'index': 0,
  'level': 1.0,
  'name': u'USECASES',
  'prodlink': u'None'},
 {'children': [[]],
  'desc': u'None',
  'domain': u'USECASES',
  'imglink': u'None',
  'index': 1,
  'level': 2.0,
  'name': u'Loading of beverages',
  'prodlink': u'None'},
 {'children': [[]],
  'desc': u'None',
  'domain': u'USECASES',
  'imglink': u'None',
  'index': 2,
  'level': 2.0,
  'name': u'Unloading of beverages',
  'prodlink': u'None'},
 {'children': [[]],
  'desc': u'None',
  'domain': u'USECASES',
  'imglink': u'None',
  'index': 3,
  'level': 2.0,
  'name': u'Driving with load',


In [347]:
#prepare string for export to JSON
jsonstr="{  \"nodes\": ["

In [348]:
#print nodes
nodesstr=''
for node in newnodes:
    print "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\"}," 
    nodesstr = nodesstr + "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\",\"desc\": \"" + node["desc"] + "\",\"imglink\": \"" + node["imglink"] + "\",\"prodlink\": \"" + node["prodlink"] + "\"}," 
nodesstr

u'{"name": "USECASES","level": 1,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Loading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Unloading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving with load","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "General misuse","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving unloaded back","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Transporting Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Charging Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Disassembled transport","level": 2,"domain": "usecases","desc": "None","imglin

In [349]:
jsonstr = jsonstr + nodesstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"links\": ["
jsonstr

u'{  "nodes": [{"name": "USECASES","level": 1,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Loading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Unloading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving with load","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "General misuse","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving unloaded back","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Transporting Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Charging Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Disassembled transport","level": 2,"domain": "usecases","desc": "

In [350]:
#generate links
links = []
for node in newnodes:
    #print { "source": "Day and night", "target": "Priority A"  },
    #print node["children"]
    for child in node["children"][0]:
        #print child
        link = {}
        link["source"] = str(node["name"])
        link["target"] = child
        links.append(link)
        #print link
linkstr = ''
for link in links:
    linkstr = linkstr + "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  
    print "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  


In [351]:
jsonstr = jsonstr + linkstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"traces\": ["
jsonstr

u'{  "nodes": [{"name": "USECASES","level": 1,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Loading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Unloading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving with load","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "General misuse","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving unloaded back","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Transporting Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Charging Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Disassembled transport","level": 2,"domain": "usecases","desc": "

In [352]:
#print traces
tracestr = ''
for rowindex, row in enumerate(mdmdata):
    #print row
    for colindex, cell in enumerate(row):
        #if (cell > 0) and (cell != 'None') and (rowindex > 1) and (colindex > 1) and (rowindex != colindex):
        if (cell > 0) and (cell != 'None') and (rowindex > 3) and (colindex > 3) and (rowindex != colindex):
            #print cell, rowindex-2, colindex-1
            tracestr = tracestr + "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            print "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            #print newnodes[rowindex-4]["name"], newnodes[colindex-3]["name"] 

In [353]:
jsonstr = jsonstr + tracestr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "] }"
jsonstr

u'{  "nodes": [{"name": "USECASES","level": 1,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Loading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Unloading of beverages","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving with load","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "General misuse","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Driving unloaded back","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Transporting Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Charging Pluto","level": 2,"domain": "usecases","desc": "None","imglink": "None","prodlink": "None"},{"name": "Disassembled transport","level": 2,"domain": "usecases","desc": "

In [354]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [355]:
filename = "Nov12_2019.json"
with open(filename, 'w') as outfile:  
    outfile.write(jsonstr)